In [1]:
import wrds, datetime
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

conn = wrds.Connection()

Loading library list...
Done


# DATA COLLECTION

## SNP , RFR , Trading Days

In [2]:
query_start_date = '1990-01-01'

In [3]:
sp500_data = conn.raw_sql(f"""
SELECT
    caldt AS date,
    sprtrn AS daily_return -- Equal-Weighted Return (includes distributions) (ewretd)
FROM
    crspq.dsp500
WHERE
    caldt >= '{query_start_date}'
ORDER BY
    caldt;
""")
sp500_data['date'] = pd.to_datetime(sp500_data['date'])

sp500_data

,date,daily_return
0,1990-01-02,0.017799
1,1990-01-03,-0.002586
2,1990-01-04,-0.008613
3,1990-01-05,-0.009756
4,1990-01-08,0.004514
...,...,...
8748,2024-09-24,0.002511
8749,2024-09-25,-0.001861
8750,2024-09-26,0.004039
8751,2024-09-27,-0.001253


In [4]:
risk_free_data = conn.raw_sql(f'''
    SELECT
        date,
        rf AS daily_rf_rate -- One Month Treasury Bill Rate (daily)
    FROM
        ff.factors_daily
    WHERE
        date >= '{query_start_date}'
''')

risk_free_data['date'] = pd.to_datetime(risk_free_data['date'])

risk_free_data

,date,daily_rf_rate
0,1990-01-02,0.00026
1,1990-01-03,0.00026
2,1990-01-04,0.00026
3,1990-01-05,0.00026
4,1990-01-08,0.00026
...,...,...
8771,2024-10-25,0.00017
8772,2024-10-28,0.00017
8773,2024-10-29,0.00017
8774,2024-10-30,0.00017


In [5]:
last_trading_days = conn.raw_sql(f'''
SELECT DISTINCT
    MAX(date) AS last_trade_date
FROM crsp.dsf
WHERE date >= '{query_start_date}'
GROUP BY DATE_TRUNC('month', date)
ORDER BY last_trade_date
''')

last_trading_days['last_trade_date'] = pd.to_datetime(last_trading_days['last_trade_date'])
last_trading_days['month'] = last_trading_days['last_trade_date'].dt.month
last_trading_days['year'] = last_trading_days['last_trade_date'].dt.year
last_trading_day_mapping = last_trading_days.set_index(['year', 'month'])['last_trade_date'].to_dict()

## Choosing Companies

In [6]:
# retrieves ID (permno) and earliest recorded name for each company in CRSP database
# 37,776 companies available

comps = conn.raw_sql( '''
SELECT permno, MIN(comnam) AS company_name
FROM crsp.stocknames
GROUP BY permno
''')
comps

,permno,company_name
0,83264,GREIF BROTHERS CORP
1,63618,HINDERLITER ENERGY EQUIP CORP
2,10896,CAMILLE ST MORITZ INC
3,69906,SEIBELS BRUCE GROUP INC
4,79030,GREAT CENTRAL MINES LTD
...,...,...
37771,14886,ARK E T F TRUST
37772,79163,A M F M INC
37773,86036,BRENTWOOD INSTRUMENTS INC
37774,92970,CHINA EDUCATION ALLIANCE INC


In [7]:
#TODO: choose a way to narrow the above list of permnos to <500. Will then use that list with the following functions to gather fin data
#this will go away when the above is completed.

company_search = 'AMAZON' # FIND A COMPANY HERE

comps[comps['company_name'].str.contains(f'{company_search}')]

,permno,company_name
9682,84788,AMAZON COM INC


## Get Unique Identifiers

In [8]:
def get_gvkey(permno):
    link = conn.raw_sql(f'''
    SELECT *
    FROM crsp.ccmxpf_linktable
    WHERE lpermno = {permno};
    ''')
    
    return link['gvkey'][0]

In [9]:
#permnos = [10107, 86580, 84788]
# BELOW IS THE LIST OF PERMNOs FOR TEN COMPANIES

query = """
select distinct on (a.permno) a.permno, a.permco, a.cusip, a.comnam
from crsp.stocknames a
where a.comnam ilike '%%alphabet%%' and a.cusip like '02079K30'
    or a.comnam ilike '%%amazon%%'
    or a.comnam ilike 'apple inc%%'
    or a.comnam ilike '%%meta platforms%%'
    or a.comnam ilike '%%microsoft%%'
    or a.comnam ilike '%%nvidia%%'
    or a.comnam ilike '%%tesla inc%%'
    or a.comnam ilike '%%netflix%%'
    or a.comnam ilike '%%walmart%%'
    or a.comnam ilike '%%pfizer%%'
;
"""

ids = conn.raw_sql(query)
permnos = ids['permno'].to_list()

gvkeys = [get_gvkey(permno) for permno in permnos]
ids['gvkey'] = gvkeys

ids

,permno,permco,cusip,comnam,gvkey
0,10107,8048,59491810,MICROSOFT CORP,012141
1,13407,54084,30303M10,META PLATFORMS INC,170617
2,14593,7,03783310,APPLE INC,001690
3,21936,21394,71708110,PFIZER CHAS & CO INC,008530
4,55976,21880,93114210,WALMART INC,011259
5,84788,15473,02313510,AMAZON COM INC,064768
6,86580,16382,67066G10,NVIDIA CORP,117768
7,89393,43145,64110L10,NETFLIX INC,147579
8,90319,45483,02079K30,ALPHABET INC,160329
9,93436,53453,88160R10,TESLA INC,184996


In [10]:
chosen_index = 6

## Query Company Data

In [11]:
def get_company_financials(gvkey):
    company_fin_data = conn.raw_sql(f'''
    SELECT
        datadate AS date,
        gvkey AS gvkey,
        rdq AS reporting_date, -- Date of which information was reported
        atq AS total_assets,  -- Total Assets
        chq AS cash_holdings,  -- Cash and Short-Term Investments
        dlttq + dlcq AS total_debt,  -- Total Debt (long-term + short-term debt)
        ibq AS earnings,  -- Earnings before extraordinary items
        xrdq AS rd_expense,  -- R&D expense
        dvpq AS dividends_paid,  -- Dividends paid
        xintq AS interest_expense  -- Interest expense
    FROM
        comp.fundq
    WHERE
        gvkey = '{gvkey}'
        AND datadate >= '{query_start_date}'  -- Ensure data is after the link start date
    ''')
    
    company_fin_data['date'] = pd.to_datetime(company_fin_data['date'])
    company_fin_data['reporting_date'] = pd.to_datetime(company_fin_data['reporting_date'])
    
    return company_fin_data.dropna()

In [12]:
def make_PiT_ffill(company_fin_data):

    temp = company_fin_data.copy()
    
    #company_fin_data['month_start'] = company_fin_data['date'] - pd.offsets.MonthBegin()
    #company_fin_data['month_end'] = company_fin_data['date']

    temp['reporting_date'] = pd.to_datetime(temp['reporting_date']) + pd.Timedelta(days=1)
    temp['month'] = temp['reporting_date'].dt.month
    temp['year'] = temp['reporting_date'].dt.year
    
    temp['EOM_trade_date'] = temp.apply(
        lambda row: last_trading_day_mapping.get((row['year'], row['month'])),
        axis=1
    )
    temp.drop(columns=['date','month','year','reporting_date'], axis=1, inplace=True)
    temp.rename(columns={'EOM_trade_date':'date'}, inplace=True)
    temp = temp[temp.columns[-1:].append(temp.columns[:-1])]
    temp['date'] = pd.to_datetime(temp['date'])

    start_date = temp['date'].min()
    end_date = temp['date'].max()

    all_months = pd.date_range(start=start_date, end=end_date, freq='ME')
    full_range_df = pd.DataFrame({'date': all_months})

    temp = pd.merge(full_range_df, temp, on='date', how='left')

    temp = temp.ffill()

    temp['month_start'] = temp['date'] - pd.offsets.MonthBegin()
    temp['month_end'] = temp['date']

    return temp

In [13]:
def get_prices(permno):
    company_stock_prcs = conn.raw_sql(f'''
    SELECT
        permno,
        date,
        prc AS stock_price,
        prc * shrout AS market_cap  -- Market cap
    FROM
        crsp.dsf
    WHERE
        permno = {permno}
    ''')
    
    company_stock_prcs['date'] = pd.to_datetime(company_stock_prcs['date'])

    return company_stock_prcs

In [14]:
def get_comp_data(permno, gvkey):

    company_fin_data = make_PiT_ffill(get_company_financials(gvkey))
    company_stock_prcs = get_prices(permno)
    
    company = pd.merge(company_fin_data, company_stock_prcs, on='date', how='left').ffill()

    company['stock_return'] = company['stock_price'].pct_change()

    return company

In [15]:
# EXAMPLE USAGE

example = get_comp_data(ids['permno'][chosen_index], ids['gvkey'][chosen_index])

example

,date,gvkey,total_assets,cash_holdings,total_debt,earnings,rd_expense,dividends_paid,interest_expense,month_start,month_end,permno,stock_price,market_cap,stock_return
0,2007-05-31,117768,2800.868,678.951,0.0,132.259,158.321,0.0,0.0,2007-05-01,2007-05-31,86580.0,34.63900,1.257094e+07,NaN
1,2007-06-30,117768,2800.868,678.951,0.0,132.259,158.321,0.0,0.0,2007-06-01,2007-06-30,86580.0,34.63900,1.257094e+07,0.000000
2,2007-07-31,117768,2800.868,678.951,0.0,132.259,158.321,0.0,0.0,2007-07-01,2007-07-31,86580.0,45.76000,1.674244e+07,0.321054
3,2007-08-31,117768,3036.267,914.745,0.0,172.732,157.952,0.0,0.0,2007-08-01,2007-08-31,86580.0,51.16000,1.871816e+07,0.118007
4,2007-09-30,117768,3036.267,914.745,0.0,172.732,157.952,0.0,0.0,2007-09-01,2007-09-30,86580.0,51.16000,1.871816e+07,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,2023-07-31,117768,44460.000,5079.000,12080.0,2043.000,1875.000,0.0,66.0,2023-07-01,2023-07-31,86580.0,467.29001,1.154206e+09,0.104652
195,2023-08-31,117768,49555.000,5783.000,10954.0,6189.000,2041.000,0.0,65.0,2023-08-01,2023-08-31,86580.0,493.54999,1.219068e+09,0.056196
196,2023-09-30,117768,49555.000,5783.000,10954.0,6189.000,2041.000,0.0,65.0,2023-09-01,2023-09-30,86580.0,493.54999,1.219068e+09,0.000000
197,2023-10-31,117768,49555.000,5783.000,10954.0,6189.000,2041.000,0.0,65.0,2023-10-01,2023-10-31,86580.0,407.79999,1.005635e+09,-0.173741


# Calculate Cash Hedged Returns

In [16]:
def calculate_b_it(company):
    data = company.copy()
    data['market_cap_t_minus_1'] = data['market_cap'].shift() # paper uses M_{t-1} for the denoms
    data['leverage'] = data['total_debt'] / (data['total_debt'] + data['market_cap'])
    
    # Y VALUES
    data['r_minus_R'] = data['stock_return'] - data['rf_rate']
    
    # REGRESSION VARIABLES
    data['gamma_1'] = (data['cash_holdings'].diff()) / data['market_cap_t_minus_1']
    data['gamma_2'] = (data['earnings'].diff()) / data['market_cap_t_minus_1']
    data['gamma_3'] = ((data['total_assets'] - data['cash_holdings']).diff()) / data['market_cap_t_minus_1']
    data['gamma_4'] = (data['rd_expense'].diff()) / data['market_cap_t_minus_1']
    data['gamma_5'] = (data['interest_expense'].diff()) / data['market_cap_t_minus_1']
    data['gamma_6'] = (data['dividends_paid'].diff()) / data['market_cap_t_minus_1']
    data['gamma_7']= data['cash_holdings_t_minus_1'] / data['market_cap_t_minus_1']
    data['gamma_8'] = data['leverage']
    data['gamma_9'] = (data['total_debt'].diff() + data['market_cap_t_minus_1'].diff()) / (data['total_debt'].shift() + data['market_cap_t_minus_1'].shift())
    data['gamma_10'] = (data['market_cap_t_minus_1'] * (data['cash_holdings'].diff())) / (data['market_cap'] ** 2)
    data['gamma_11'] = (data['leverage'] * (data['cash_holdings'].diff())) / data['market_cap']
    
    data = data.dropna()
    
    y = data['r_minus_R']
    X = data[['gamma_1', 'gamma_2', 'gamma_3', 'gamma_4', 'gamma_5', 'gamma_6', 'gamma_7', 'gamma_8', 'gamma_9', 'gamma_10', 'gamma_11']]
    
    X = sm.add_constant(X)
    
    model = sm.OLS(y, X).fit()
    
    #print(model.summary())
    
    data['marginal_cash_value'] = (
        model.params.iloc[0] +
        (model.params.iloc[10] * (data['cash_holdings_t_minus_1'] / data['market_cap_t_minus_1'])) +
        (model.params.iloc[11] * data['leverage'])
    )
    
    data['average_cash_value'] = data['marginal_cash_value'] * data['cash_holdings']
    
    company['b_it'] = data['average_cash_value'].pct_change()  # monthly cash return

In [17]:
def calculate_e_it(company):
    company['cash_hedged_return'] = (1 / (1 - company['cash_share_weight'])) * (company['stock_return'] - (company['cash_share_weight']  * company['b_it']))

In [18]:
def winsorize(series, lower=0.01, upper=0.99):
    lower_threshold = series.quantile(lower)
    upper_threshold = series.quantile(upper)
    return series.clip(lower=lower_threshold, upper=upper_threshold)

In [19]:
def calculate_returns(company):
    company['cash_share_weight'] = (company['cash_holdings'] / company['total_assets'])
    company['cash_holdings_t_minus_1'] = company['cash_holdings'].shift()
    
    calculate_b_it(company)
    calculate_e_it(company)
    
    company['cash_hedged_return'] = winsorize(company['cash_hedged_return'])

In [20]:
def calculate_rolling_beta(stock_return, market_return, window = 65):
    rolling_cov = stock_return.rolling(window).cov(market_return)
    rolling_var = market_return.rolling(window).var()
    return rolling_cov / rolling_var

# Data Aggregation

In [ ]:
def main():

    agg_fin_data = pd.DataFrame(columns=['date', 'gvkey', 'reporting_date', 'total_assets', 'cash_holdings',
        'total_debt', 'earnings', 'rd_expense', 'dividends_paid',
        'interest_expense', 'stock_price', 'stock_return', 'market_cap',
        'snp_return', 'rf_rate', 'cash_share_weight', 'cash_holdings_t_minus_1',
        'b_it', 'cash_hedged_return', 'company_beta', 'cash_hedged_beta', 'comnam'])

    # Create a mapping from permno to comnam
    comnam_mapping = ids.set_index('permno')['comnam'].to_dict()

    for i in range(len(permnos)):
        company = get_comp_data(permnos[i], gvkeys[i])
        
        # Add the company name (comnam) to the company DataFrame
        company['comnam'] = comnam_mapping.get(permnos[i], 'Unknown')
        
        def aggregate_monthly(start_date, end_date, daily_data, column, agg_func):
            mask = (daily_data['date'] >= start_date) & (daily_data['date'] <= end_date)
            return agg_func(daily_data.loc[mask, column])
        
        company['snp_return'] = company.apply(
            lambda row: aggregate_monthly(row['month_start'], row['month_end'], sp500_data, 'daily_return', 
                                        lambda x: np.prod(1 + x) - 1), axis=1
        )
        
        company['rf_rate'] = company.apply(
            lambda row: aggregate_monthly(row['month_start'], row['month_end'], risk_free_data, 'daily_rf_rate', 
                                        lambda x: np.prod(1 + x) - 1), axis=1
        )
        
        company.drop(columns=['month_start', 'month_end'], axis=1, inplace=True)
        
        calculate_returns(company)
        
        company['company_beta'] = calculate_rolling_beta(company['stock_return'], company['snp_return'])
        company['cash_hedged_beta'] = calculate_rolling_beta(company['cash_hedged_return'], company['snp_return'])
        
        if not agg_fin_data.empty:
            agg_fin_data = pd.concat([agg_fin_data, company], ignore_index=True)
        else:
            agg_fin_data = company
        
    agg_fin_data = agg_fin_data.sort_values(by=['comnam', 'date']).reset_index(drop=True)

    agg_fin_data = agg_fin_data[['date', 'comnam', 'gvkey', 'permno', 'cash_hedged_return', 'company_beta', 'cash_hedged_beta', 'snp_return', 'rf_rate']].dropna()

    return agg_fin_data

out = main()

out

,date,comnam,gvkey,permno,cash_hedged_return,company_beta,cash_hedged_beta,snp_return,rf_rate
67,2012-08-31,APPLE INC,001690,14593.0,0.106838,1.042542,1.918554,0.019763,0.000000
68,2012-09-30,APPLE INC,001690,14593.0,0.004101,1.012342,1.868360,0.024235,0.000000
69,2012-10-31,APPLE INC,001690,14593.0,-0.134853,0.999512,1.827361,-0.019790,0.000000
70,2012-11-30,APPLE INC,001690,14593.0,-0.050023,0.997610,1.837826,0.002845,0.000000
71,2012-12-31,APPLE INC,001690,14593.0,-0.097732,1.009654,1.863459,0.007071,0.000200
...,...,...,...,...,...,...,...,...,...
1859,2023-06-30,TESLA INC,184996,93436.0,0.407915,0.887093,1.274930,0.064727,0.003998
1860,2023-07-31,TESLA INC,184996,93436.0,0.040041,0.882093,1.271031,0.031139,0.004409
1861,2023-08-31,TESLA INC,184996,93436.0,-0.049392,0.889406,1.281454,-0.017717,0.004610
1862,2023-09-30,TESLA INC,184996,93436.0,-0.000009,0.873825,1.259573,-0.048720,0.004208
